Required Software

In [ ]:
pip install -r Basics.txt

1. Getting The Data

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('netflix_customer_churn.csv')
print(df.head())

In [ ]:
df.columns

In [ ]:
df.describe

In [ ]:
df.isna().sum()

In [ ]:
# introucing missing values for the sake of demonstration
import random
df.loc[df.sample(frac=0.01).index, 'device'] = np.nan

In [ ]:
df.isna().sum()

In [ ]:
#Calculate the percentage
df.isna().sum() / df.shape[0]*100

In [ ]:
# Check for duplicate rows (entire row is identical)
print("Total duplicates:", df.duplicated().sum())

# Check for duplicates in specific columns (e.g., 'customer_id' should be unique)
print("Duplicate customer IDs:", df['customer_id'].duplicated().sum())

# Display duplicate rows (if any)
duplicates = df[df.duplicated(keep=False)]  # keep=False marks all duplicates
print("Sample duplicates:\n", duplicates.head(2))



In [ ]:
# Drop all duplicate rows (keeping the first occurrence)
df = df.drop_duplicates()

# Verify duplicates are gone
print("Duplicates after removal:", df.duplicated().sum())

2. Setting up the database


In [ ]:
df.dtypes

In [ ]:
import sqlite3
conn = sqlite3.connect('churn.db')

In [ ]:
cursor = conn.cursor()

CREATE TABLE customers (
   CREATE TABLE customers (
    customerID VARCHAR(36) PRIMARY KEY,
    age SMALLINT UNSIGNED,
    gender ENUM('Female', 'Male', 'Other'),
    subscriptionType ENUM('Basic', 'Standard', 'Premium'),
    watchHours DECIMAL(6, 2),
    lastLoginDays SMALLINT UNSIGNED,
    region VARCHAR(30),
    device VARCHAR(20),
    monthlyFee DECIMAL(10, 2),
    churn BOOLEAN NOT NULL,
    paymentMethod VARCHAR(30),
    numberOfProfiles TINYINT UNSIGNED,
    avgWatchTimePerDay DECIMAL(5, 2),
    favoriteGenre VARCHAR(30)
);

);


In [ ]:

# Connect to database (or create one)
conn = sqlite3.connect("churn.db")
cursor = conn.cursor()

# Create table (Note: SQLite doesn't support ENUM directly)
cursor.execute("""
CREATE TABLE customers (
    customerID TEXT PRIMARY KEY,
    age INTEGER,
    gender TEXT,
    subscriptionType TEXT,
    watchHours REAL,
    lastLoginDays INTEGER,
    region TEXT,
    device TEXT,
    monthlyFee REAL,
    churn BOOLEAN NOT NULL,
    paymentMethod TEXT,
    numberOfProfiles INTEGER,
    avgWatchTimePerDay REAL,
    favoriteGenre TEXT
);
""")

conn.commit()



In [ ]:
cursor.execute("PRAGMA table_info(customers);")
cursor.fetchall()

In [ ]:
for index, row in df.iterrows():
    cursor.execute('''
        INSERT INTO customers (
            customerID,
            age,
            gender, 
            subscriptionType,
            watchHours, 
            lastLoginDays, 
            region, 
            device,
            monthlyFee, 
            churn, 
            paymentMethod,
            numberOfProfiles, 
            avgWatchTimePerDay, 
            favoriteGenre
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        row['customer_id'],
        row['age'],
        row['gender'],
        row['subscription_type'],
        row['watch_hours'],
        row['last_login_days'],
        row['region'],
        row['device'],
        row['monthly_fee'],
        bool(row['churned']),  # Convert int (0/1) to boolean
        row['payment_method'],
        row['number_of_profiles'],
        row['avg_watch_time_per_day'],
        row['favorite_genre']
    ))

conn.commit()


In [ ]:
df.columns

In [ ]:
def run_query(query):
    cursor.execute(query)
    return cursor.fetchall()

In [ ]:
query = '''
SELECT gender, region, AVG(device) AS device
FROM customers
GROUP BY gender

'''
run_query(query)

In [ ]:
query = '''
SELECT gender, region, paymentMethod, device, AVG(avgWatchTimePerDay) AS avgWatchTimePerDay
FROM customers
GROUP BY gender, age

'''
run_query(query)

Churn Analysis


In [ ]:
query = '''

SELECT AVG(avgWatchTimePerDay) AS avgWatchTimePerDay, AVG(MonthlyFee) AS avg_monthly_fee,  paymentMethod, device, region, COUNT(*) AS churn_count
FROM customers
WHERE Churn = 1
GROUP BY paymentMethod, device, region
ORDER BY churn_count DESC;
'''
run_query(query)

More Analysis Later


More Detailed Analysis Using Python


In [ ]:
churn_by_gender = df.groupby('gender') ['churned'].sum()/df.shape[0]
churn_by_gender

In [ ]:
churn_by_device = df.groupby('device') ['churned'].sum()/df.shape[0]
churn_by_device

In [ ]:
churn_by_payment_method = df.groupby('payment_method')['churned'].sum() / df.shape[0]
churn_by_payment_method

Data Visualization


In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns

# Churn by Contract Type

sns.countplot(data=df, x='device', hue='churned') 
plt.title('Churn Rate by Device Type')

plt.show()

In [ ]:
sns.countplot(data=df, x='payment_method', hue='churned') 
plt.title('Churn Rate by Payment Method Type')

plt.show()

In [ ]:
sns.countplot(data=df, x='payment_method', hue='churned')
plt.title('Churn Rate by Payment Method Type')

plt.show()

Machine Learning


In [ ]:
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer



Logic Regression

In [ ]:
# Example data preparation
X = df.drop('churned', axis=1)
y = df['churned']

# Identify column types
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Preprocessing pipelines
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numerical_features),
    ('cat', categorical_pipeline, categorical_features)
])

# Final pipeline
model_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit and evaluate
model_pipeline.fit(X_train, y_train)
y_pred = model_pipeline.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier
X = df.drop('churned', axis=1)
y = df['churned']

# Identify column types
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Numeric preprocessing
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Categorical preprocessing
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing
preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numerical_features),
    ('cat', categorical_pipeline, categorical_features)
])

# Full pipeline with Decision Tree
model_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('classifier', DecisionTreeClassifier(max_depth=5, random_state=42))  # you can tune max_depth
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model_pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = model_pipeline.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Random Forest Classifier


In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Load and split your data
X = df.drop('churned', axis=1)
y = df['churned']

# Identify categorical and numerical columns
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Preprocessing for numeric features
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical features
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessor
preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numerical_features),
    ('cat', categorical_pipeline, categorical_features)
])

# Full pipeline with Random Forest
model_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, max_depth=8, random_state=42))
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model_pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = model_pipeline.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))